In [ ]:
import pandas as pd
df = pd.read_table("guillaume.txt", sep=";")
df.head()

In [ ]:
len(df)

# Nettoyage 

On supprime la ligne 1956360 qui comporte comme valeur pour chaque colonne l'en-tête

In [ ]:
df.drop(1956360, axis=0, inplace=True)

On change le type des variables quantitatives

In [ ]:
var_quanti = ["Montant","ScoringFP1","ScoringFP2","ScoringFP3","TauxImpNb_RB","TauxImpNB_CPM","DiffDateTr1","DiffDateTr2","DiffDateTr3","CA3TRetMtt","CA3TR"]
var_quanti_bug = ["VerifianceCPT1","VerifianceCPT2","VerifianceCPT3","EcartNumCheq","NbrMagasin3J","D2CB"]
new_df=df

for i in var_quanti_bug:
    new_df[i] = pd.to_numeric(new_df[i], downcast="float")
# On change le type des données
for i in var_quanti:
    new_df[i] = new_df[i].str.replace(",",".")
    new_df[i] = pd.to_numeric(new_df[i], downcast="float")

In [ ]:
import datetime  

date = []
heure = []


for i in new_df["DateTransaction"]:
    split = i.split(" ")
    date.append(split[0])
    heure.append(split[1])

for i in range(0,len(date)):
    date[i] = datetime.datetime.strptime(date[i], '%Y-%m-%d')



In [ ]:
new_df["DateTransaction"]=date
new_df["Heure"]=heure

On s'occupe des des valeurs manquantes 

In [ ]:
print(new_df.isna().sum())

Aucune valeur manquantes

On nettoie la colonne FlagImpaye

In [ ]:
new_df.FlagImpaye.value_counts()

In [ ]:
new_df.FlagImpaye=new_df.FlagImpaye.replace(0,"0")
new_df.FlagImpaye=new_df.FlagImpaye.replace(1,"1")
new_df.FlagImpaye.value_counts(normalize=True)

In [ ]:
Y_train = train_df.FlagImpaye
Y_test = test_df.FlagImpaye
X_train = train_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
X_test = test_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
len(Y_train[Y_train=="1"])

# SAMPLING

In [ ]:
train_df=new_df[(new_df["DateTransaction"]>='2017-02-01') & (new_df["DateTransaction"]<='2017-08-31')]
test_df=new_df[(new_df["DateTransaction"]>'2017-08-31')]

In [ ]:
Y_train = train_df.FlagImpaye
Y_test = test_df.FlagImpaye
X_train = train_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
X_test = test_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
len(Y_train[Y_train=="1"])

# MACHINE LEARNING

## UNDERSAMPLING : 50-50

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=1)
rus.fit(X_train, Y_train)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)
len(X_train_resampled)

In [ ]:
Y_train_resampled.value_counts(normalize=True)

In [ ]:
best_f = 0

### Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 24, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 5, step = 1)}

In [ ]:
model_arbre = DecisionTreeClassifier()
f1 = make_scorer(f1_score , average='macro')
model_arbre = GridSearchCV(model_arbre,
                                  parameters,
                                  scoring = f1,
                                  verbose = 2, 
                                  cv = 5)
model_arbre.fit(X_train_resampled, Y_train_resampled)

In [ ]:
print("Voici les paramètres du meilleure modèle : " + 
      str(model_arbre.best_estimator_))
print("Voici le "  + str(model_arbre.scorer_) + 
      " du meilleure modèle : " + str(model_arbre.best_score_))

In [ ]:
Y_pred = model_arbre.predict(X_test)
Y_pred_proba = model_arbre.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_arbre.best_estimator_
    best_f = F

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 21, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 4, step = 1)}


In [ ]:
modele_foret = RandomForestClassifier()
f1 = make_scorer(f1_score , average='macro')
modele_foret = GridSearchCV(modele_foret,
                                  parameters,
                                  scoring = f1,
                                  verbose = 2, 
                                  cv = 3)
modele_foret.fit(X_train_resampled, Y_train_resampled)

In [ ]:
print("Voici les paramètres du meilleure modèle : " + 
      str(modele_foret.best_estimator_))
print("Voici le "  + str(modele_foret.scorer_) + 
      " du meilleure modèle : " + str(modele_foret.best_score_))

In [ ]:
Y_pred = modele_foret.predict(X_test)
Y_pred_proba = modele_foret.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)
if F>best_f :
    best_model = modele_foret.best_estimator_
    best_f = F

### SVM

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn("SVC")

In [ ]:
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_svm.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_svm
    best_f = F

### Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_adaboost = AdaBoostClassifier(best_model)
model_adaboost.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_adaboost.predict(X_test)
Y_pred_proba = model_adaboost.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
model_bagging = BaggingClassifier()

model_bagging.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_bagging.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,Y_pred))

In [ ]:
from sklearn.metrics import f1_score
print('f1_score : ' + 
      str(f1_score(Y_test,Y_pred, average='macro')))

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

## UNDERSAMPLING : 0.9 - 0.1 sur 100 000 individus

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy={"0": 99000, "1": 1000})
rus.fit(X_train, Y_train)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)
len(X_train_resampled)

In [ ]:
Y_train_resampled.value_counts(normalize=True)

In [ ]:
best_f = 0

### Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 24, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 5, step = 1)}

In [ ]:
model_arbre = DecisionTreeClassifier()
f1 = make_scorer(f1_score , average='macro')
model_arbre = GridSearchCV(model_arbre,
                                  parameters,
                                  scoring = f1,
                                  verbose = 2, 
                                  cv = 5)
model_arbre.fit(X_train_resampled, Y_train_resampled)

In [ ]:
print("Voici les paramètres du meilleure modèle : " + 
      str(model_arbre.best_estimator_))
print("Voici le "  + str(model_arbre.scorer_) + 
      " du meilleure modèle : " + str(model_arbre.best_score_))

In [ ]:
Y_pred = model_arbre.predict(X_test)
Y_pred_proba = model_arbre.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_arbre.best_estimator_
    best_f = F

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 21, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 4, step = 1)}

In [ ]:
modele_foret = RandomForestClassifier()
f1 = make_scorer(f1_score , average='macro')
modele_foret = GridSearchCV(modele_foret,
                                  parameters,
                                  scoring = f1,
                                  verbose = 2, 
                                  cv = 3)
modele_foret.fit(X_train_resampled, Y_train_resampled)

In [ ]:
print("Voici les paramètres du meilleure modèle : " + 
      str(modele_foret.best_estimator_))
print("Voici le "  + str(modele_foret.scorer_) + 
      " du meilleure modèle : " + str(modele_foret.best_score_))

In [ ]:
Y_pred = modele_foret.predict(X_test)
Y_pred_proba = modele_foret.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)
if F>best_f :
    best_model = modele_foret.best_estimator_
    best_f = F

### SVM

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn("SVC")

In [ ]:
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_svm.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_svm
    best_f = F

### Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_adaboost = AdaBoostClassifier(best_model)
model_adaboost.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_adaboost.predict(X_test)
Y_pred_proba = model_adaboost.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
model_bagging = BaggingClassifier()

model_bagging.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_bagging.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

## UNDERSAMPLING : 0.9-0.1 en prenant tous les Y

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=0.01)
rus.fit(X_train, Y_train)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)
len(X_train_resampled)

In [ ]:
Y_train_resampled.value_counts(normalize=True)

In [ ]:
best_f = 0

### Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_arbre = DecisionTreeClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=2)
model_arbre.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_arbre.predict(X_test)
Y_pred_proba = model_arbre.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_arbre
    best_f = F

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_forest = RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=2)
model_forest.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = modele_foret.predict(X_test)
Y_pred_proba = modele_foret.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)
if F>best_f :
    best_model = modele_foret
    best_f = F

### SVM

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn("SVC")

In [ ]:
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_svm.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_svm
    best_f = F

### Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_adaboost = AdaBoostClassifier(best_model)
model_adaboost.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_adaboost.predict(X_test)
Y_pred_proba = model_adaboost.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
model_bagging = BaggingClassifier()

model_bagging.fit(X_train_resampled, Y_train_resampled)

In [ ]:
Y_pred = model_bagging.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

In [ ]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)